In [12]:
# Standalone RL training cell for YOUR model (leg_hopper.xml)
# - TWO-DOF actuation: hip_motor + knee_motor controlled by the policy
# - Passive tendon spring: F = k*(L-L0) when L>L0 else 0
# - Reward = "best repeatable hop height": encourages multiple hops reaching similar apex height
#
# Requirements:
#   pip install mujoco gymnasium stable-baselines3 numpy imageio imageio-ffmpeg

import os
import numpy as np
import mujoco
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor, VecVideoRecorder


# ------------------------- USER SETTINGS -------------------------
MODEL_XML_PATH = "leg_hopper.xml"

# Joint/actuator names in your XML
HIP_JOINT_NAME       = "hip_joint"
KNEE_JOINT_NAME      = "knee_joint"
HIP_MOTOR_NAME       = "hip_motor"
KNEE_MOTOR_NAME      = "knee_motor"
TENDON_NAME          = "leg_spring"
TENDON_ACT_NAME      = "leg_spring_actuator"

# Root slider joints (in your XML)
ROOT_X_JOINT_NAME    = "hip_linear_x"
ROOT_Z_JOINT_NAME    = "hip_linear_z"

# Tendon spring parameters (your values)
SPRING_K  = 500.0       # N/m
SPRING_L0 = 0.2109      # m

# Action scaling (must respect each actuator ctrlrange in XML)
# If ctrlrange for hip/knee is [-1, 1], keep these = 1.0.
MAX_HIP_CTRL  = 2.0
MAX_KNEE_CTRL = 2.0

# Env timing
SEED = 0
CTRL_HZ = 50
EPISODE_SECONDS = 3.0    # allow multiple hops

# Repeatable hop reward settings
APEX_WINDOW = 10            # how many recent hops define "repeatable height"
MIN_HOPS_FOR_SCORE = 2     # only start scoring after you have at least this many apex events
CONSISTENCY_W = 4.0        # penalty weight on hop-to-hop variation (higher = more consistent)
HEIGHT_W = 5.0             # reward weight on repeatable height per apex
FLIP_TERMINATE_ANGLE = 2.2 # rad, safety termination if hip joint goes crazy

# PPO
TOTAL_TIMESTEPS = 300_000
SAVE_PATH = "ppo_hip_knee_repeatable_hops.zip"

# Video recording
RECORD_VIDEO = True
VIDEO_DIR = "videos"
VIDEO_EVERY_N_STEPS = 50_000
VIDEO_SECONDS = 20.0
VIDEO_LENGTH = int(VIDEO_SECONDS * CTRL_HZ)

# ---------------------------------------------------------------


def _name2id_or_raise(model, obj_type, name: str) -> int:
    idx = mujoco.mj_name2id(model, obj_type, name)
    if idx < 0:
        raise ValueError(f"Name not found: {name} (type={obj_type})")
    return int(idx)

def _joint_qposadr(model, joint_id: int) -> int:
    return int(model.jnt_qposadr[joint_id])

def _joint_dofadr(model, joint_id: int) -> int:
    return int(model.jnt_dofadr[joint_id])


class HipKneeRepeatHopEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 50}

    def __init__(
        self,
        xml_path: str,
        render_mode: str | None = None,
        episode_seconds: float = 3.0,
        ctrl_hz: int = 50,
        max_hip_ctrl: float = 1.0,
        max_knee_ctrl: float = 1.0,
        seed: int = 0,
    ):
        super().__init__()

        if not os.path.exists(xml_path):
            raise FileNotFoundError(f"MuJoCo XML not found: {xml_path}")

        self.model = mujoco.MjModel.from_xml_path(xml_path)
        self.data = mujoco.MjData(self.model)

        # IDs
        self.hip_jid   = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, HIP_JOINT_NAME)
        self.knee_jid  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, KNEE_JOINT_NAME)
        self.rootx_jid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, ROOT_X_JOINT_NAME)
        self.rootz_jid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, ROOT_Z_JOINT_NAME)

        self.hip_aid   = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, HIP_MOTOR_NAME)
        self.knee_aid  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, KNEE_MOTOR_NAME)

        self.tendon_id  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_TENDON, TENDON_NAME)
        self.tendon_aid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, TENDON_ACT_NAME)
        
        self.track_body_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "hip_linear_guide")

        # Addresses
        self.hip_qpos_adr   = _joint_qposadr(self.model, self.hip_jid)
        self.hip_dof_adr    = _joint_dofadr(self.model, self.hip_jid)
        self.knee_qpos_adr  = _joint_qposadr(self.model, self.knee_jid)
        self.knee_dof_adr   = _joint_dofadr(self.model, self.knee_jid)
        self.rootz_qpos_adr = _joint_qposadr(self.model, self.rootz_jid)
        self.rootz_dof_adr  = _joint_dofadr(self.model, self.rootz_jid)

        # Timing
        self.episode_seconds = float(episode_seconds)
        self.ctrl_hz = int(ctrl_hz)
        self.ctrl_dt = 1.0 / self.ctrl_hz
        self.sim_dt = float(self.model.opt.timestep)
        self.substeps = max(1, int(round(self.ctrl_dt / self.sim_dt)))

        self.max_hip_ctrl  = float(max_hip_ctrl)
        self.max_knee_ctrl = float(max_knee_ctrl)
        self.render_mode = render_mode
        self.np_random, _ = gym.utils.seeding.np_random(seed)

        self.horizon_steps = int(round(self.episode_seconds * self.ctrl_hz))
        self.t = 0

        # Hop bookkeeping
        self._prev_vz = 0.0
        self._apex_heights = []

        # Spaces
        # obs: [hip_q, hip_qd, knee_q, knee_qd, root_z, root_vz, tendon_L, ncon, phase]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(9,), dtype=np.float32)

        # action: [hip_u, knee_u] in [-1, 1] each (scaled by MAX_*_CTRL)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)

        # Rendering (rgb_array)
        self._renderer = None

    # ------------------ state helpers ------------------
    def _get_root_z_vz(self) -> tuple[float, float]:
        z  = float(self.data.qpos[self.rootz_qpos_adr])
        vz = float(self.data.qvel[self.rootz_dof_adr])
        return z, vz

    def _get_knee_q_qd(self) -> tuple[float, float]:
        return float(self.data.qpos[self.knee_qpos_adr]), float(self.data.qvel[self.knee_dof_adr])

    def _get_hip_q_qd(self) -> tuple[float, float]:
        return float(self.data.qpos[self.hip_qpos_adr]), float(self.data.qvel[self.hip_dof_adr])

    def _get_tendon_length(self) -> float:
        return float(self.data.ten_length[self.tendon_id])

    def _get_obs(self) -> np.ndarray:
        hip_q, hip_qd = self._get_hip_q_qd()
        knee_q, knee_qd = self._get_knee_q_qd()
        z, vz = self._get_root_z_vz()
        L = self._get_tendon_length()
        ncon = float(self.data.ncon)
        phase = float(self.t) / float(max(1, self.horizon_steps))
        return np.array([hip_q, hip_qd, knee_q, knee_qd, z, vz, L, ncon, phase], dtype=np.float32)

    # ------------------ tendon spring ------------------
    def _apply_tendon_spring_force(self):
        L = float(self.data.ten_length[self.tendon_id])
        F = SPRING_K * (L - SPRING_L0) if L > SPRING_L0 else 0.0
        self.data.ctrl[self.tendon_aid] = float(F)
        return L, float(F)

    # ------------------ hop event detection ------------------
    def _detect_apex_event(self, z: float, vz: float, ncon: int) -> bool:
        apex = (ncon == 0) and (self._prev_vz > 0.0) and (vz <= 0.0)
        self._prev_vz = vz
        if apex:
            self._apex_heights.append(z)
        return apex

    def _repeatable_height_score(self) -> tuple[float, float]:
        if len(self._apex_heights) < MIN_HOPS_FOR_SCORE:
            return 0.0, 0.0
        recent = self._apex_heights[-APEX_WINDOW:]
        repeatable_h = float(np.min(recent))
        std_recent = float(np.std(recent)) if len(recent) >= 2 else 0.0
        return repeatable_h, std_recent

    # ------------------ gym API ------------------
    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.np_random, _ = gym.utils.seeding.np_random(seed)

        mujoco.mj_resetData(self.model, self.data)

        # Root start
        self.data.qpos[self.rootz_qpos_adr] = 0.0
        self.data.qvel[self.rootz_dof_adr]  = 0.0

        # Small randomization on both joints (helps robustness)
        self.data.qpos[self.hip_qpos_adr]  += self.np_random.uniform(-0.05, 0.05)
        self.data.qvel[self.hip_dof_adr]   += self.np_random.uniform(-0.2, 0.2)
        self.data.qpos[self.knee_qpos_adr] += self.np_random.uniform(-0.05, 0.05)
        self.data.qvel[self.knee_dof_adr]  += self.np_random.uniform(-0.2, 0.2)

        mujoco.mj_forward(self.model, self.data)

        self.t = 0
        _, vz = self._get_root_z_vz()
        self._prev_vz = vz
        self._apex_heights = []

        return self._get_obs(), {}

    def step(self, action):
        self.t += 1
        self.data.ctrl[:] = 0.0

        # Tendon spring
        L, F = self._apply_tendon_spring_force()

        # Policy actions
        hip_u  = float(np.clip(action[0], -1.0, 1.0)) * self.max_hip_ctrl
        knee_u = float(np.clip(action[1], -1.0, 1.0)) * self.max_knee_ctrl

        self.data.ctrl[self.hip_aid]  = hip_u
        self.data.ctrl[self.knee_aid] = knee_u

        # Physics
        for _ in range(self.substeps):
            mujoco.mj_step(self.model, self.data)

        # State
        z, vz = self._get_root_z_vz()
        ncon = int(self.data.ncon)

        # Reward shaping
        reward = 0.02 * float(z)

        apex = self._detect_apex_event(z, vz, ncon)
        repeatable_h, std_recent = self._repeatable_height_score()

        if apex and len(self._apex_heights) >= MIN_HOPS_FOR_SCORE:
            reward += HEIGHT_W * repeatable_h
            reward -= CONSISTENCY_W * std_recent

        # Control penalty (both joints)
        reward -= 1e-2 * (hip_u * hip_u + knee_u * knee_u)

        # Termination / truncation
        truncated = (self.t >= self.horizon_steps)
        terminated = False

        # Safety: NaNs or extreme fall
        if not np.isfinite(z) or not np.isfinite(vz):
            terminated = True
        if z < -1.5:
            terminated = True

        # Extra safety: hip joint shouldn't blow up
        hip_q, _ = self._get_hip_q_qd()
        if abs(hip_q) > FLIP_TERMINATE_ANGLE:
            terminated = True

        info = {
            "root_z": z,
            "root_vz": vz,
            "tendon_L": L,
            "tendon_F": F,
            "hip_u": hip_u,
            "knee_u": knee_u,
            "ncon": ncon,
            "apex": bool(apex),
            "apex_count": len(self._apex_heights),
            "repeatable_h": repeatable_h,
            "std_recent": std_recent,
            "apex_heights": self._apex_heights.copy(),
        }

        return self._get_obs(), float(reward), terminated, truncated, info

    # ---- Rendering for VecVideoRecorder ----
    def render(self):
        if self.render_mode is None:
            return None
        if self.render_mode == "rgb_array":
            if self._renderer is None:
                self._renderer = mujoco.Renderer(self.model, height=480, width=640)
            self._renderer.update_scene(self.data, camera="close_cam")
            return self._renderer.render()
        if self.render_mode == "human":
            return None
        raise ValueError(f"Unknown render_mode: {self.render_mode}")

    def close(self):
        if self._renderer is not None:
            try:
                self._renderer.close()
            except Exception:
                pass
            self._renderer = None


# -------------------- Sanity check --------------------
def sanity_check(xml_path: str, steps: int = 300):
    print("=== SANITY CHECK ===")
    env = HipKneeRepeatHopEnv(
        xml_path=xml_path,
        render_mode=None,
        episode_seconds=2.0,
        ctrl_hz=CTRL_HZ,
        max_hip_ctrl=MAX_HIP_CTRL,
        max_knee_ctrl=MAX_KNEE_CTRL,
        seed=SEED
    )
    obs, _ = env.reset(seed=SEED)

    # simple open-loop to verify signals remain sane
    f = 2.0
    A = 0.5
    for i in range(steps):
        t = i / CTRL_HZ
        hip_u  = A * np.sin(2*np.pi*f*t)
        knee_u = A * np.cos(2*np.pi*f*t)
        obs, rew, terminated, truncated, info = env.step(np.array([hip_u, knee_u], dtype=np.float32))

        if i % 50 == 0:
            print(
                f"step={i:4d} | z={info['root_z']:7.4f} | vz={info['root_vz']:7.4f} | "
                f"L={info['tendon_L']:7.4f} | F={info['tendon_F']:7.2f} | "
                f"hip_u={info['hip_u']:6.3f} | knee_u={info['knee_u']:6.3f} | ncon={info['ncon']} | "
                f"apex={int(info['apex'])} | apex_count={info['apex_count']} | repeatable_h={info['repeatable_h']:.4f}"
            )

        if terminated or truncated:
            break

    env.close()
    print("Sanity check done.\n")


def make_env(render_mode=None):
    return HipKneeRepeatHopEnv(
        xml_path=MODEL_XML_PATH,
        render_mode=render_mode,
        episode_seconds=EPISODE_SECONDS,
        ctrl_hz=CTRL_HZ,
        max_hip_ctrl=MAX_HIP_CTRL,
        max_knee_ctrl=MAX_KNEE_CTRL,
        seed=SEED,
    )


# -------------------- 1) Sanity check --------------------
sanity_check(MODEL_XML_PATH, steps=300)

# -------------------- 2) Vector env + video --------------------
base_env = DummyVecEnv([lambda: make_env(render_mode="rgb_array" if RECORD_VIDEO else None)])
vec_env = VecMonitor(base_env)

if RECORD_VIDEO:
    os.makedirs(VIDEO_DIR, exist_ok=True)

    def video_trigger(step_count: int) -> bool:
        return (step_count == 0) or (step_count % VIDEO_EVERY_N_STEPS == 0)

    vec_env = VecVideoRecorder(
        vec_env,
        video_folder=VIDEO_DIR,
        record_video_trigger=video_trigger,
        video_length=VIDEO_LENGTH,
        name_prefix="ppo_hip_knee_repeat_hops"
    )

# -------------------- 3) Train PPO --------------------
model = PPO(
    policy="MlpPolicy",
    env=vec_env,
    n_steps=1024,
    batch_size=256,
    gae_lambda=0.95,
    gamma=0.99,
    n_epochs=10,
    learning_rate=3e-4,
    clip_range=0.2,
    ent_coef=0.0,
    vf_coef=0.5,
    max_grad_norm=0.5,
    verbose=1,
    seed=SEED,
)

model.learn(total_timesteps=TOTAL_TIMESTEPS)
model.save(SAVE_PATH)
print(f"Saved trained policy to: {SAVE_PATH}")

vec_env.close()
print(f"Videos (if enabled) saved in: {VIDEO_DIR}")

# -------------------- 4) Evaluation with hop stats --------------------
eval_env = make_env(render_mode=None)
best_repeatable = -np.inf

for ep in range(10):
    obs, _ = eval_env.reset(seed=SEED + ep)
    done = False
    last_info = None
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = eval_env.step(action)
        done = terminated or truncated
        last_info = info

    apex_heights = last_info["apex_heights"]
    if len(apex_heights) >= MIN_HOPS_FOR_SCORE:
        recent = apex_heights[-APEX_WINDOW:]
        repeatable = float(np.min(recent))
        best_repeatable = max(best_repeatable, repeatable)
        print(f"Episode {ep+1}: apex_count={len(apex_heights)} repeatable_h(min last {APEX_WINDOW})={repeatable:.4f}  recent={np.round(recent,4)}")
    else:
        print(f"Episode {ep+1}: apex_count={len(apex_heights)} (not enough hops to score)")

print(f"Best repeatable height over 10 eval episodes: {best_repeatable:.4f}")

eval_env.close()


=== SANITY CHECK ===
step=   0 | z=-0.0073 | vz=-0.6992 | L= 0.2114 | F=   0.25 | hip_u= 0.000 | knee_u= 1.000 | ncon=0 | apex=0 | apex_count=0 | repeatable_h=0.0000
step=  50 | z=-0.0650 | vz=-0.5275 | L= 0.2137 | F=   1.40 | hip_u=-0.000 | knee_u= 1.000 | ncon=0 | apex=0 | apex_count=4 | repeatable_h=-0.0566
Sanity check done.

Using cpu device
Saving video to g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-0-to-step-1000.mp4
MoviePy - Building video g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-0-to-step-1000.mp4.
MoviePy - Writing video g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-0-to-step-1000.mp4



MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-0-to-step-1000.mp4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 150      |
|    ep_rew_mean     | -19.6    |
| time/              |          |
|    fps             | 39       |
|    iterations      | 1        |
|    time_elapsed    | 26       |
|    total_timesteps | 1024     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 150         |
|    ep_rew_mean          | -20.2       |
| time/                   |             |
|    fps                  | 61          |
|    iterations           | 2           |
|    time_elapsed         | 33          |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.004810926 |
|    clip_fraction        |

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-50000-to-step-51000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 150         |
|    ep_rew_mean          | -11.8       |
| time/                   |             |
|    fps                  | 129         |
|    iterations           | 50          |
|    time_elapsed         | 394         |
|    total_timesteps      | 51200       |
| train/                  |             |
|    approx_kl            | 0.005344462 |
|    clip_fraction        | 0.0569      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.49       |
|    explained_variance   | 0.7         |
|    learning_rate        | 0.0003      |
|    loss                 | 0.102       |
|    n_updates            | 490         |
|    policy_gradient_loss | -0.00869    |
|    std                  

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-100000-to-step-101000.mp4
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 150          |
|    ep_rew_mean          | -9.15        |
| time/                   |              |
|    fps                  | 149          |
|    iterations           | 99           |
|    time_elapsed         | 679          |
|    total_timesteps      | 101376       |
| train/                  |              |
|    approx_kl            | 0.0076941014 |
|    clip_fraction        | 0.0895       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.12        |
|    explained_variance   | 0.54         |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0381       |
|    n_updates            | 980          |
|    policy_gradient_loss | -0.0132      |
|    

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-150000-to-step-151000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 150         |
|    ep_rew_mean          | -6.49       |
| time/                   |             |
|    fps                  | 148         |
|    iterations           | 148         |
|    time_elapsed         | 1022        |
|    total_timesteps      | 151552      |
| train/                  |             |
|    approx_kl            | 0.007217131 |
|    clip_fraction        | 0.0592      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.65       |
|    explained_variance   | 0.676       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00658    |
|    n_updates            | 1470        |
|    policy_gradient_loss | -0.0125     |
|    std                

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-200000-to-step-201000.mp4
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 150          |
|    ep_rew_mean          | -3.76        |
| time/                   |              |
|    fps                  | 152          |
|    iterations           | 197          |
|    time_elapsed         | 1326         |
|    total_timesteps      | 201728       |
| train/                  |              |
|    approx_kl            | 0.0069635026 |
|    clip_fraction        | 0.0521       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.961       |
|    explained_variance   | 0.576        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0105       |
|    n_updates            | 1960         |
|    policy_gradient_loss | -0.00809     |
|    

KeyboardInterrupt: 

In [ ]:
import time
import numpy as np
import mujoco.viewer
from stable_baselines3 import PPO

# This 2nd cell is designed to work *directly* with your 1st cell:
# - Uses the same HipKneeRepeatHopEnv class (defined above)
# - Uses the same constants: SAVE_PATH, MODEL_XML_PATH, CTRL_HZ, MAX_HIP_CTRL, MAX_KNEE_CTRL, SEED, FLIP_TERMINATE_ANGLE
# - Runs indefinitely (episode_seconds=1e9) and resets on failure

def run_policy_continuous_viewer_paced(
    *,
    policy,
    xml_path: str,
    ctrl_hz: int,
    max_hip_ctrl: float,
    max_knee_ctrl: float,
    seed: int,
    flip_terminate_angle: float,
    speed: float = 0.25,          # 1.0 = real-time, 0.25 = 4x slower
    reset_on_failure: bool = True,
    print_every_seconds: float = 5.0,
    max_real_fps: float = 60.0,
):
    speed = float(max(1e-6, speed))

    # Use YOUR env class from cell 1
    env = HipKneeRepeatHopEnv(
        xml_path=xml_path,
        render_mode=None,          # viewer handles visualization directly
        episode_seconds=1e9,       # effectively "run forever"
        ctrl_hz=ctrl_hz,
        max_hip_ctrl=max_hip_ctrl,
        max_knee_ctrl=max_knee_ctrl,
        seed=seed,
    )

    try:
        obs, _ = env.reset(seed=seed)

        ctrl_step = 0
        sim_dt_per_ctrl = 1.0 / float(ctrl_hz)

        # Real-time pacing (slow down or speed up the wall-clock)
        target_real_dt_per_ctrl = sim_dt_per_ctrl / speed
        min_real_dt = 1.0 / float(max_real_fps)
        target_real_dt_per_ctrl = max(target_real_dt_per_ctrl, min_real_dt)

        print_every = max(1, int(round(print_every_seconds * ctrl_hz)))

        with mujoco.viewer.launch_passive(env.model, env.data) as viewer:
            while viewer.is_running():
                loop_t0 = time.perf_counter()

                action, _ = policy.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = env.step(action)
                ctrl_step += 1

                viewer.sync()

                if (ctrl_step % print_every) == 0:
                    print(
                        f"ctrl={ctrl_step:7d} | z={info.get('root_z', float('nan')):+.4f} | "
                        f"vz={info.get('root_vz', float('nan')):+.4f} | "
                        f"ncon={info.get('ncon', -1)} | apex_cnt={info.get('apex_count', -1)} | "
                        f"repeat_h={info.get('repeatable_h', float('nan')):.4f} | "
                        f"std={info.get('std_recent', float('nan')):.4f} | "
                        f"hip_u={info.get('hip_u', float('nan')):+.3f} | "
                        f"knee_u={info.get('knee_u', float('nan')):+.3f}"
                    )

                # Reset conditions (terminated OR truncated, though truncated shouldn't happen with 1e9)
                if terminated or truncated:
                    # Try to infer a useful reason (matches logic in your env.step)
                    hip_q, _ = env._get_hip_q_qd()
                    root_z = info.get("root_z", np.nan)
                    root_vz = info.get("root_vz", np.nan)

                    reason = "terminated" if terminated else "truncated"
                    if (not np.isfinite(root_z)) or (not np.isfinite(root_vz)):
                        reason = "NaN/Inf state"
                    elif root_z < -1.5:
                        reason = "fell below z < -1.5"
                    elif abs(hip_q) > flip_terminate_angle:
                        reason = f"hip angle blew up |hip_q|={abs(hip_q):.3f} > {flip_terminate_angle:.3f}"

                    print(f"[RESET] {reason} at ctrl={ctrl_step}")

                    if reset_on_failure:
                        obs, _ = env.reset()
                        ctrl_step = 0
                    else:
                        break

                # Wall-clock pacing
                elapsed = time.perf_counter() - loop_t0
                sleep_for = target_real_dt_per_ctrl - elapsed
                if sleep_for > 0:
                    time.sleep(sleep_for)

    finally:
        env.close()


# ---- usage: assumes cell 1 already ran and produced these names ----
# SAVE_PATH, MODEL_XML_PATH, CTRL_HZ, MAX_HIP_CTRL, MAX_KNEE_CTRL, SEED, FLIP_TERMINATE_ANGLE
policy = PPO.load(SAVE_PATH)

run_policy_continuous_viewer_paced(
    policy=policy,
    xml_path=MODEL_XML_PATH,
    ctrl_hz=CTRL_HZ,
    max_hip_ctrl=MAX_HIP_CTRL,
    max_knee_ctrl=MAX_KNEE_CTRL,
    seed=SEED,
    flip_terminate_angle=FLIP_TERMINATE_ANGLE,
    speed=0.2,
)


In [15]:
# Standalone RL training cell for YOUR model (leg_hopper.xml)
# - TWO-DOF actuation: hip_motor + knee_motor controlled by the policy
# - Passive tendon spring: F = k*(L-L0) when L>L0 else 0
# - Reward = forward hopping: make progress in +x while maintaining hopping/airtime and reasonable height
#
# Requirements:
#   pip install mujoco gymnasium stable-baselines3 numpy imageio imageio-ffmpeg

import os
import numpy as npF
import mujoco
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor, VecVideoRecorder


# ------------------------- USER SETTINGS -------------------------
MODEL_XML_PATH = "leg_hopper.xml"

# Joint/actuator names in your XML
HIP_JOINT_NAME       = "hip_joint"
KNEE_JOINT_NAME      = "knee_joint"
HIP_MOTOR_NAME       = "hip_motor"
KNEE_MOTOR_NAME      = "knee_motor"
TENDON_NAME          = "leg_spring"
TENDON_ACT_NAME      = "leg_spring_actuator"

# Root slider joints (in your XML)
ROOT_X_JOINT_NAME    = "hip_linear_x"
ROOT_Z_JOINT_NAME    = "hip_linear_z"

# Tendon spring parameters (your values)
SPRING_K  = 500.0       # N/m
SPRING_L0 = 0.2109      # m

# Action scaling (must respect each actuator ctrlrange in XML)
MAX_HIP_CTRL  = 2.0
MAX_KNEE_CTRL = 2.0

# Env timing
SEED = 0
CTRL_HZ = 50
EPISODE_SECONDS = 4.0    # give it a bit more time to build a gait

# Forward hopping reward settings
FORWARD_W = 2.0            # reward weight on forward progress per step (dx)
AIRTIME_W = 0.2            # reward for being airborne (ncon==0)
HEIGHT_W  = 0.05           # small shaping toward being above ground
BACKWARD_PEN = 4.0         # penalize negative dx harder
SIDEWAYS_VX_PEN = 0.01     # penalize very high vx (keeps things sane)
CTRL_PEN = 1e-2            # control penalty (u^2)

# Optional hop event shaping (kept light)
APEX_BONUS_W = 0.2          # bonus each apex event (airborne peak)
MIN_Z_FOR_APEX = 0.02       # ignore tiny "apex" jiggles near ground

FLIP_TERMINATE_ANGLE = 2.2  # rad, safety termination if hip joint goes crazy

# PPO
TOTAL_TIMESTEPS = 400_000
SAVE_PATH = "ppo_forward_hops.zip"

# Video recording
RECORD_VIDEO = True
VIDEO_DIR = "videos"
VIDEO_EVERY_N_STEPS = 50_000
VIDEO_SECONDS = 20.0
VIDEO_LENGTH = int(VIDEO_SECONDS * CTRL_HZ)

# ---------------------------------------------------------------


def _name2id_or_raise(model, obj_type, name: str) -> int:
    idx = mujoco.mj_name2id(model, obj_type, name)
    if idx < 0:
        raise ValueError(f"Name not found: {name} (type={obj_type})")
    return int(idx)

def _joint_qposadr(model, joint_id: int) -> int:
    return int(model.jnt_qposadr[joint_id])

def _joint_dofadr(model, joint_id: int) -> int:
    return int(model.jnt_dofadr[joint_id])


class ForwardHopEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 50}

    def __init__(
        self,
        xml_path: str,
        render_mode: str | None = None,
        episode_seconds: float = 4.0,
        ctrl_hz: int = 50,
        max_hip_ctrl: float = 1.0,
        max_knee_ctrl: float = 1.0,
        seed: int = 0,
    ):
        super().__init__()

        if not os.path.exists(xml_path):
            raise FileNotFoundError(f"MuJoCo XML not found: {xml_path}")

        self.model = mujoco.MjModel.from_xml_path(xml_path)
        self.data = mujoco.MjData(self.model)

        # IDs
        self.hip_jid   = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, HIP_JOINT_NAME)
        self.knee_jid  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, KNEE_JOINT_NAME)
        self.rootx_jid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, ROOT_X_JOINT_NAME)
        self.rootz_jid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, ROOT_Z_JOINT_NAME)

        self.hip_aid   = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, HIP_MOTOR_NAME)
        self.knee_aid  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, KNEE_MOTOR_NAME)

        self.tendon_id  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_TENDON, TENDON_NAME)
        self.tendon_aid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, TENDON_ACT_NAME)

        # Addresses
        self.hip_qpos_adr   = _joint_qposadr(self.model, self.hip_jid)
        self.hip_dof_adr    = _joint_dofadr(self.model, self.hip_jid)
        self.knee_qpos_adr  = _joint_qposadr(self.model, self.knee_jid)
        self.knee_dof_adr   = _joint_dofadr(self.model, self.knee_jid)

        self.rootx_qpos_adr = _joint_qposadr(self.model, self.rootx_jid)
        self.rootx_dof_adr  = _joint_dofadr(self.model, self.rootx_jid)
        self.rootz_qpos_adr = _joint_qposadr(self.model, self.rootz_jid)
        self.rootz_dof_adr  = _joint_dofadr(self.model, self.rootz_jid)

        # Timing
        self.episode_seconds = float(episode_seconds)
        self.ctrl_hz = int(ctrl_hz)
        self.ctrl_dt = 1.0 / self.ctrl_hz
        self.sim_dt = float(self.model.opt.timestep)
        self.substeps = max(1, int(round(self.ctrl_dt / self.sim_dt)))

        self.max_hip_ctrl  = float(max_hip_ctrl)
        self.max_knee_ctrl = float(max_knee_ctrl)
        self.render_mode = render_mode
        self.np_random, _ = gym.utils.seeding.np_random(seed)

        self.horizon_steps = int(round(self.episode_seconds * self.ctrl_hz))
        self.t = 0

        # Bookkeeping for progress + apex detection
        self._prev_x = 0.0
        self._prev_vz = 0.0
        self._apex_count = 0

        # Spaces
        # obs: [hip_q, hip_qd, knee_q, knee_qd, x, vx, z, vz, tendon_L, ncon, phase]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(11,), dtype=np.float32)
        # action: [hip_u, knee_u] in [-1, 1] each (scaled)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)

        self._renderer = None

    # ------------------ state helpers ------------------
    def _get_root_x_vx(self) -> tuple[float, float]:
        x  = float(self.data.qpos[self.rootx_qpos_adr])
        vx = float(self.data.qvel[self.rootx_dof_adr])
        return x, vx

    def _get_root_z_vz(self) -> tuple[float, float]:
        z  = float(self.data.qpos[self.rootz_qpos_adr])
        vz = float(self.data.qvel[self.rootz_dof_adr])
        return z, vz

    def _get_knee_q_qd(self) -> tuple[float, float]:
        return float(self.data.qpos[self.knee_qpos_adr]), float(self.data.qvel[self.knee_dof_adr])

    def _get_hip_q_qd(self) -> tuple[float, float]:
        return float(self.data.qpos[self.hip_qpos_adr]), float(self.data.qvel[self.hip_dof_adr])

    def _get_tendon_length(self) -> float:
        return float(self.data.ten_length[self.tendon_id])

    def _get_obs(self) -> np.ndarray:
        hip_q, hip_qd = self._get_hip_q_qd()
        knee_q, knee_qd = self._get_knee_q_qd()
        x, vx = self._get_root_x_vx()
        z, vz = self._get_root_z_vz()
        L = self._get_tendon_length()
        ncon = float(self.data.ncon)
        phase = float(self.t) / float(max(1, self.horizon_steps))
        return np.array([hip_q, hip_qd, knee_q, knee_qd, x, vx, z, vz, L, ncon, phase], dtype=np.float32)

    # ------------------ tendon spring ------------------
    def _apply_tendon_spring_force(self):
        L = float(self.data.ten_length[self.tendon_id])
        F = SPRING_K * (L - SPRING_L0) if L > SPRING_L0 else 0.0
        self.data.ctrl[self.tendon_aid] = float(F)
        return L, float(F)

    # ------------------ apex event detection ------------------
    def _detect_apex_event(self, z: float, vz: float, ncon: int) -> bool:
        # apex while airborne and vertical velocity crosses + -> -
        apex = (ncon == 0) and (self._prev_vz > 0.0) and (vz <= 0.0) and (z > MIN_Z_FOR_APEX)
        self._prev_vz = vz
        if apex:
            self._apex_count += 1
        return apex

    # ------------------ gym API ------------------
    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.np_random, _ = gym.utils.seeding.np_random(seed)

        mujoco.mj_resetData(self.model, self.data)

        # Root start
        self.data.qpos[self.rootx_qpos_adr] = 0.0
        self.data.qvel[self.rootx_dof_adr]  = 0.0
        self.data.qpos[self.rootz_qpos_adr] = 0.0
        self.data.qvel[self.rootz_dof_adr]  = 0.0

        # Mild randomization on joints
        self.data.qpos[self.hip_qpos_adr]  += self.np_random.uniform(-0.05, 0.05)
        self.data.qvel[self.hip_dof_adr]   += self.np_random.uniform(-0.2, 0.2)
        self.data.qpos[self.knee_qpos_adr] += self.np_random.uniform(-0.05, 0.05)
        self.data.qvel[self.knee_dof_adr]  += self.np_random.uniform(-0.2, 0.2)

        mujoco.mj_forward(self.model, self.data)

        self.t = 0
        x, _ = self._get_root_x_vx()
        z, vz = self._get_root_z_vz()
        self._prev_x = x
        self._prev_vz = vz
        self._apex_count = 0

        return self._get_obs(), {}

    def step(self, action):
        self.t += 1
        self.data.ctrl[:] = 0.0

        # Tendon spring
        L, F = self._apply_tendon_spring_force()

        # Policy actions
        hip_u  = float(np.clip(action[0], -1.0, 1.0)) * self.max_hip_ctrl
        knee_u = float(np.clip(action[1], -1.0, 1.0)) * self.max_knee_ctrl
        self.data.ctrl[self.hip_aid]  = hip_u
        self.data.ctrl[self.knee_aid] = knee_u

        # Physics
        for _ in range(self.substeps):
            mujoco.mj_step(self.model, self.data)

        # State
        x, vx = self._get_root_x_vx()
        z, vz = self._get_root_z_vz()
        ncon = int(self.data.ncon)

        # Progress
        dx = float(x - self._prev_x)
        self._prev_x = x

        # Reward
        reward = 0.0

        # Forward progress (primary)
        if dx >= 0.0:
            reward += FORWARD_W * dx
        else:
            reward += -BACKWARD_PEN * abs(dx)

        # Encourage actual hopping (airtime) lightly
        if ncon == 0:
            reward += AIRTIME_W * (1.0 / self.ctrl_hz)

        # Small height shaping to avoid face-plant "sliding"
        reward += HEIGHT_W * float(z)

        # Apex bonus (optional, light)
        apex = self._detect_apex_event(z, vz, ncon)
        if apex:
            reward += APEX_BONUS_W

        # Keep vx from exploding
        reward -= SIDEWAYS_VX_PEN * float(vx * vx)

        # Control penalty
        reward -= CTRL_PEN * (hip_u * hip_u + knee_u * knee_u)

        # Termination / truncation
        truncated = (self.t >= self.horizon_steps)
        terminated = False

        # Safety: NaNs or extreme fall
        if not np.isfinite(x) or not np.isfinite(z) or not np.isfinite(vx) or not np.isfinite(vz):
            terminated = True
        if z < -1.5:
            terminated = True

        # Extra safety: hip joint shouldn't blow up
        hip_q, _ = self._get_hip_q_qd()
        if abs(hip_q) > FLIP_TERMINATE_ANGLE:
            terminated = True

        info = {
            "root_x": x,
            "root_vx": vx,
            "root_z": z,
            "root_vz": vz,
            "dx": dx,
            "tendon_L": L,
            "tendon_F": F,
            "hip_u": hip_u,
            "knee_u": knee_u,
            "ncon": ncon,
            "apex": bool(apex),
            "apex_count": int(self._apex_count),
        }

        return self._get_obs(), float(reward), terminated, truncated, info

    # ---- Rendering for VecVideoRecorder ----
    def render(self):
        if self.render_mode is None:
            return None
        if self.render_mode == "rgb_array":
            if self._renderer is None:
                self._renderer = mujoco.Renderer(self.model, height=480, width=640)
            self._renderer.update_scene(self.data, camera="close_cam")
            return self._renderer.render()
        if self.render_mode == "human":
            return None
        raise ValueError(f"Unknown render_mode: {self.render_mode}")

    def close(self):
        if self._renderer is not None:
            try:
                self._renderer.close()
            except Exception:
                pass
            self._renderer = None


# -------------------- Sanity check --------------------
def sanity_check(xml_path: str, steps: int = 300):
    print("=== SANITY CHECK ===")
    env = ForwardHopEnv(
        xml_path=xml_path,
        render_mode=None,
        episode_seconds=2.0,
        ctrl_hz=CTRL_HZ,
        max_hip_ctrl=MAX_HIP_CTRL,
        max_knee_ctrl=MAX_KNEE_CTRL,
        seed=SEED
    )
    obs, _ = env.reset(seed=SEED)

    # open-loop with a phase offset to induce fore-aft asymmetry
    f = 2.0
    A = 0.6
    for i in range(steps):
        t = i / CTRL_HZ
        hip_u  = A * np.sin(2*np.pi*f*t + 0.3)
        knee_u = A * np.sin(2*np.pi*f*t)
        obs, rew, terminated, truncated, info = env.step(np.array([hip_u, knee_u], dtype=np.float32))

        if i % 50 == 0:
            print(
                f"step={i:4d} | x={info['root_x']:7.4f} | vx={info['root_vx']:7.4f} | "
                f"z={info['root_z']:7.4f} | vz={info['root_vz']:7.4f} | dx={info['dx']: .5f} | "
                f"L={info['tendon_L']:7.4f} | F={info['tendon_F']:7.2f} | "
                f"hip_u={info['hip_u']:6.3f} | knee_u={info['knee_u']:6.3f} | ncon={info['ncon']} | "
                f"apex_count={info['apex_count']}"
            )

        if terminated or truncated:
            break

    env.close()
    print("Sanity check done.\n")


def make_env(render_mode=None):
    return ForwardHopEnv(
        xml_path=MODEL_XML_PATH,
        render_mode=render_mode,
        episode_seconds=EPISODE_SECONDS,
        ctrl_hz=CTRL_HZ,
        max_hip_ctrl=MAX_HIP_CTRL,
        max_knee_ctrl=MAX_KNEE_CTRL,
        seed=SEED,
    )


# -------------------- 1) Sanity check --------------------
sanity_check(MODEL_XML_PATH, steps=300)

# -------------------- 2) Vector env + video --------------------
base_env = DummyVecEnv([lambda: make_env(render_mode="rgb_array" if RECORD_VIDEO else None)])
vec_env = VecMonitor(base_env)

if RECORD_VIDEO:
    os.makedirs(VIDEO_DIR, exist_ok=True)

    def video_trigger(step_count: int) -> bool:
        return (step_count == 0) or (step_count % VIDEO_EVERY_N_STEPS == 0)

    vec_env = VecVideoRecorder(
        vec_env,
        video_folder=VIDEO_DIR,
        record_video_trigger=video_trigger,
        video_length=VIDEO_LENGTH,
        name_prefix="ppo_forward_hops"
    )

# -------------------- 3) Train PPO --------------------
model = PPO(
    policy="MlpPolicy",
    env=vec_env,
    n_steps=1024,
    batch_size=256,
    gae_lambda=0.95,
    gamma=0.99,
    n_epochs=10,
    learning_rate=3e-4,
    clip_range=0.2,
    ent_coef=0.0,
    vf_coef=0.5,
    max_grad_norm=0.5,
    verbose=1,
    seed=SEED,
)

model.learn(total_timesteps=TOTAL_TIMESTEPS)
model.save(SAVE_PATH)
print(f"Saved trained policy to: {SAVE_PATH}")

vec_env.close()
print(f"Videos (if enabled) saved in: {VIDEO_DIR}")

# -------------------- 4) Evaluation with forward stats --------------------
eval_env = make_env(render_mode=None)

best_distance = -np.inf
best_speed = -np.inf

for ep in range(10):
    obs, _ = eval_env.reset(seed=SEED + ep)
    done = False
    last_info = None
    x0 = float(eval_env.data.qpos[eval_env.rootx_qpos_adr])

    steps = 0
    apex_ct = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = eval_env.step(action)
        done = terminated or truncated
        last_info = info
        steps += 1
        apex_ct = info["apex_count"]

    x1 = float(last_info["root_x"])
    dist = float(x1 - x0)
    duration = steps / float(CTRL_HZ)
    speed = dist / max(1e-9, duration)

    best_distance = max(best_distance, dist)
    best_speed = max(best_speed, speed)

    print(f"Episode {ep+1}: dist={dist:.4f} m | avg_speed={speed:.4f} m/s | apex_count={apex_ct} | final_z={last_info['root_z']:.4f}")

print(f"Best distance over 10 eval episodes: {best_distance:.4f} m")
print(f"Best avg speed over 10 eval episodes: {best_speed:.4f} m/s")

eval_env.close()


=== SANITY CHECK ===
step=   0 | x= 0.0018 | vx= 0.1707 | z=-0.0009 | vz=-0.0967 | dx= 0.00176 | L= 0.2114 | F=   0.25 | hip_u= 0.355 | knee_u= 0.000 | ncon=0 | apex_count=0
step=  50 | x=-0.8726 | vx=-1.5668 | z=-0.0909 | vz=-0.2011 | dx=-0.03321 | L= 0.2256 | F=   7.36 | hip_u= 0.355 | knee_u=-0.000 | ncon=0 | apex_count=0
Sanity check done.

Using cpu device
Saving video to g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-0-to-step-1000.mp4
MoviePy - Building video g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-0-to-step-1000.mp4.
MoviePy - Writing video g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-0-to-step-1000.mp4



MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-0-to-step-1000.mp4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -12.3    |
| time/              |          |
|    fps             | 48       |
|    iterations      | 1        |
|    time_elapsed    | 21       |
|    total_timesteps | 1024     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -11.5        |
| time/                   |              |
|    fps                  | 71           |
|    iterations           | 2            |
|    time_elapsed         | 28           |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0048388382 |
|    clip_fraction      

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-50000-to-step-51000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -0.0303     |
| time/                   |             |
|    fps                  | 152         |
|    iterations           | 50          |
|    time_elapsed         | 336         |
|    total_timesteps      | 51200       |
| train/                  |             |
|    approx_kl            | 0.012470712 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.26       |
|    explained_variance   | 0.51        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0181     |
|    n_updates            | 490         |
|    policy_gradient_loss | -0.019      |
|    std                  | 0.747 

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-100000-to-step-101000.mp4
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | 3.23         |
| time/                   |              |
|    fps                  | 173          |
|    iterations           | 99           |
|    time_elapsed         | 585          |
|    total_timesteps      | 101376       |
| train/                  |              |
|    approx_kl            | 0.0067111896 |
|    clip_fraction        | 0.081        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.56        |
|    explained_variance   | 0.507        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0211      |
|    n_updates            | 980          |
|    policy_gradient_loss | -0.0144      |
|    std     

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-150000-to-step-151000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | 6.75        |
| time/                   |             |
|    fps                  | 170         |
|    iterations           | 148         |
|    time_elapsed         | 888         |
|    total_timesteps      | 151552      |
| train/                  |             |
|    approx_kl            | 0.007025549 |
|    clip_fraction        | 0.0806      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.929      |
|    explained_variance   | 0.61        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0166     |
|    n_updates            | 1470        |
|    policy_gradient_loss | -0.0125     |
|    std                  | 0.38

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-200000-to-step-201000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | 9.8         |
| time/                   |             |
|    fps                  | 164         |
|    iterations           | 197         |
|    time_elapsed         | 1229        |
|    total_timesteps      | 201728      |
| train/                  |             |
|    approx_kl            | 0.013267456 |
|    clip_fraction        | 0.179       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.371      |
|    explained_variance   | 0.392       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0186     |
|    n_updates            | 1960        |
|    policy_gradient_loss | -0.0168     |
|    std                  | 0.29

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-250000-to-step-251000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | 11.7        |
| time/                   |             |
|    fps                  | 160         |
|    iterations           | 246         |
|    time_elapsed         | 1574        |
|    total_timesteps      | 251904      |
| train/                  |             |
|    approx_kl            | 0.020797653 |
|    clip_fraction        | 0.172       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.193       |
|    explained_variance   | 0.625       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0121     |
|    n_updates            | 2450        |
|    policy_gradient_loss | -0.00906    |
|    std                  | 0.21

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-300000-to-step-301000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | 13.2        |
| time/                   |             |
|    fps                  | 162         |
|    iterations           | 294         |
|    time_elapsed         | 1855        |
|    total_timesteps      | 301056      |
| train/                  |             |
|    approx_kl            | 0.009705661 |
|    clip_fraction        | 0.136       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.713       |
|    explained_variance   | 0.69        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00401    |
|    n_updates            | 2930        |
|    policy_gradient_loss | -0.011      |
|    std                  | 0.16

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-350000-to-step-351000.mp4
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 200        |
|    ep_rew_mean          | 14.5       |
| time/                   |            |
|    fps                  | 162        |
|    iterations           | 343        |
|    time_elapsed         | 2155       |
|    total_timesteps      | 351232     |
| train/                  |            |
|    approx_kl            | 0.01784015 |
|    clip_fraction        | 0.196      |
|    clip_range           | 0.2        |
|    entropy_loss         | 1.24       |
|    explained_variance   | 0.67       |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0215    |
|    n_updates            | 3420       |
|    policy_gradient_loss | -0.00577   |
|    std                  | 0.13       |
|    value

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\Optimization_Simulation\mujoco_leg_hopper\videos\ppo_forward_hops-step-400000-to-step-401000.mp4
Videos (if enabled) saved in: videos
Episode 1: dist=6.0023 m | avg_speed=1.5006 m/s | apex_count=60 | final_z=0.0323
Episode 2: dist=6.0450 m | avg_speed=1.5113 m/s | apex_count=57 | final_z=0.0398
Episode 3: dist=5.9663 m | avg_speed=1.4916 m/s | apex_count=58 | final_z=0.0458
Episode 4: dist=5.9768 m | avg_speed=1.4942 m/s | apex_count=59 | final_z=0.0281
Episode 5: dist=5.8577 m | avg_speed=1.4644 m/s | apex_count=61 | final_z=0.0305
Episode 6: dist=5.9643 m | avg_speed=1.4911 m/s | apex_count=59 | final_z=0.0445
Episode 7: dist=6.0182 m | avg_speed=1.5046 m/s | apex_count=59 | final_z=0.0447
Episode 8: dist=5.6555 m | avg_speed=1.4139 m/s | apex_count=62 | final_z=0.0342
Episode 9: dist=5.8791 m | avg_speed=1.4698 m/s | apex_count=60 | final_z=0.0350
Episode 10: dist=5.8607 m | avg_speed=1.4652 m/s | apex_count=60

In [20]:
import time
import numpy as np
import mujoco.viewer
from stable_baselines3 import PPO

# This viewer cell is designed to work *directly* with your ForwardHopEnv training cell:
# - Uses the same ForwardHopEnv class (defined above)
# - Uses the same constants: SAVE_PATH, MODEL_XML_PATH, CTRL_HZ, MAX_HIP_CTRL, MAX_KNEE_CTRL, SEED, FLIP_TERMINATE_ANGLE
# - Runs indefinitely (episode_seconds=1e9) and resets on failure

def run_forward_policy_continuous_viewer_paced(
    *,
    policy,
    xml_path: str,
    ctrl_hz: int,
    max_hip_ctrl: float,
    max_knee_ctrl: float,
    seed: int,
    flip_terminate_angle: float,
    speed: float = 0.01,          # 1.0 = real-time, 0.25 = 4x slower
    reset_on_failure: bool = True,
    print_every_seconds: float = 5.0,
    max_real_fps: float = 100.0,
):
    speed = float(max(1e-6, speed))

    # Use YOUR env class from the forward-hopping cell
    env = ForwardHopEnv(
        xml_path=xml_path,
        render_mode=None,          # viewer handles visualization directly
        episode_seconds=1e9,       # effectively "run forever"
        ctrl_hz=ctrl_hz,
        max_hip_ctrl=max_hip_ctrl,
        max_knee_ctrl=max_knee_ctrl,
        seed=seed,
    )

    try:
        obs, _ = env.reset(seed=seed)

        ctrl_step = 0
        sim_dt_per_ctrl = 1.0 / float(ctrl_hz)

        # Real-time pacing (slow down or speed up the wall-clock)
        target_real_dt_per_ctrl = sim_dt_per_ctrl / speed
        min_real_dt = 1.0 / float(max_real_fps)
        target_real_dt_per_ctrl = max(target_real_dt_per_ctrl, min_real_dt)

        print_every = max(1, int(round(print_every_seconds * ctrl_hz)))

        with mujoco.viewer.launch_passive(env.model, env.data) as viewer:
            while viewer.is_running():
                loop_t0 = time.perf_counter()

                action, _ = policy.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = env.step(action)
                ctrl_step += 1

                viewer.sync()

                if (ctrl_step % print_every) == 0:
                    # distance since last reset is tracked via env._prev_x logic; we can also just print current x
                    print(
                        f"ctrl={ctrl_step:7d} | x={info.get('root_x', float('nan')):+.4f} | "
                        f"vx={info.get('root_vx', float('nan')):+.4f} | "
                        f"z={info.get('root_z', float('nan')):+.4f} | "
                        f"vz={info.get('root_vz', float('nan')):+.4f} | "
                        f"dx={info.get('dx', float('nan')):+.5f} | "
                        f"ncon={info.get('ncon', -1)} | apex_cnt={info.get('apex_count', -1)} | "
                        f"hip_u={info.get('hip_u', float('nan')):+.3f} | "
                        f"knee_u={info.get('knee_u', float('nan')):+.3f} | "
                        f"L={info.get('tendon_L', float('nan')):.4f} | F={info.get('tendon_F', float('nan')):.2f}"
                    )

                # Reset conditions (terminated OR truncated, though truncated shouldn't happen with 1e9)
                if terminated or truncated:
                    # Try to infer a useful reason (matches logic in your env.step)
                    hip_q, _ = env._get_hip_q_qd()
                    root_x = info.get("root_x", np.nan)
                    root_z = info.get("root_z", np.nan)
                    root_vx = info.get("root_vx", np.nan)
                    root_vz = info.get("root_vz", np.nan)

                    reason = "terminated" if terminated else "truncated"
                    if (not np.isfinite(root_x)) or (not np.isfinite(root_z)) or (not np.isfinite(root_vx)) or (not np.isfinite(root_vz)):
                        reason = "NaN/Inf state"
                    elif root_z < -1.5:
                        reason = "fell below z < -1.5"
                    elif abs(hip_q) > flip_terminate_angle:
                        reason = f"hip angle blew up |hip_q|={abs(hip_q):.3f} > {flip_terminate_angle:.3f}"

                    print(f"[RESET] {reason} at ctrl={ctrl_step}")

                    if reset_on_failure:
                        obs, _ = env.reset()
                        ctrl_step = 0
                    else:
                        break

                # Wall-clock pacing
                elapsed = time.perf_counter() - loop_t0
                sleep_for = target_real_dt_per_ctrl - elapsed
                if sleep_for > 0:
                    time.sleep(sleep_for)

    finally:
        env.close()


# ---- usage: assumes your forward-training cell already ran and produced these names ----
# SAVE_PATH, MODEL_XML_PATH, CTRL_HZ, MAX_HIP_CTRL, MAX_KNEE_CTRL, SEED, FLIP_TERMINATE_ANGLE
policy = PPO.load(SAVE_PATH)

run_forward_policy_continuous_viewer_paced(
    policy=policy,
    xml_path=MODEL_XML_PATH,
    ctrl_hz=CTRL_HZ,
    max_hip_ctrl=MAX_HIP_CTRL,
    max_knee_ctrl=MAX_KNEE_CTRL,
    seed=SEED,
    flip_terminate_angle=FLIP_TERMINATE_ANGLE,
    speed=0.05,
)
